In [48]:
import torch
from torch import nn  


# Tensor Properties

In [31]:
tensor = torch.Tensor(
    [
        [[1,2], [3,4]],
        [[5,6], [7,8]],
        [[12,34], [9,12]],
    ]
)

In [32]:
tensor

tensor([[[ 1.,  2.],
         [ 3.,  4.]],

        [[ 5.,  6.],
         [ 7.,  8.]],

        [[12., 34.],
         [ 9., 12.]]])

In [8]:
tensor.device

device(type='cpu')

In [33]:
print(tensor.shape)
print(tensor.shape[0])
print(tensor.shape[1])
print(tensor.type)


torch.Size([3, 2, 2])
3
2
<built-in method type of Tensor object at 0x78df74196a80>


In [11]:
print(f"Rank: {len(tensor.shape)}")
print(f"Number of elements {tensor.numel()}")

Rank: 3
Number of elements 12


In [12]:
tensor[1]

tensor([[5, 6],
        [7, 8]])

## Initializing Tensors

In [13]:
torch.ones_like(tensor)

tensor([[[1, 1],
         [1, 1]],

        [[1, 1],
         [1, 1]],

        [[1, 1],
         [1, 1]]])

In [14]:
torch.zeros_like(tensor)

tensor([[[0, 0],
         [0, 0]],

        [[0, 0],
         [0, 0]],

        [[0, 0],
         [0, 0]]])

In [34]:
torch.randn_like(tensor)

tensor([[[ 0.9323, -0.3249],
         [ 1.0980,  1.9216]],

        [[-1.3921,  0.0825],
         [ 0.1673, -0.9808]],

        [[-2.0706,  0.5780],
         [-0.0597, -0.3301]]])

In [24]:
torch.randn(3,2,2, device="cpu")

tensor([[[-0.5067, -0.7791],
         [-0.0654, -1.0737]],

        [[ 0.4481,  0.1183],
         [ 0.3308,  0.2367]],

        [[ 0.7938, -0.0925],
         [-0.1894,  0.5915]]])

In [25]:
(tensor - 5) * 2

tensor([[[-8, -6],
         [-4, -2]],

        [[ 0,  2],
         [ 4,  6]],

        [[14, 58],
         [ 8, 14]]])

In [37]:
print(f"Mean {tensor.mean()}")
print(f"Std: {tensor.std()}")
print(f"Along some direction only: {tensor.mean(0)}")

Mean 8.583333015441895
Std: 8.764165878295898
Along some direction only: tensor([[ 6.0000, 14.0000],
        [ 6.3333,  8.0000]])


In [39]:
linear = nn.Linear(10,2)
example_input = torch.randn(4, 10)
example_output = linear(example_input)
example_output

tensor([[ 0.0070,  1.3981],
        [ 0.5824,  0.2040],
        [-0.6463,  0.2239],
        [-0.4834,  0.4623]], grad_fn=<AddmmBackward0>)

In [42]:
relu = nn.ReLU()
relu_output = relu(example_output)
print(relu_output)
print(relu_output.mean())

tensor([[0.0070, 1.3981],
        [0.5824, 0.2040],
        [0.0000, 0.2239],
        [0.0000, 0.4623]], grad_fn=<ReluBackward0>)
tensor(0.3597, grad_fn=<MeanBackward0>)


In [43]:
batchnorm = nn.BatchNorm1d(2)
batchnorm_output = batchnorm(relu_output)
print(batchnorm_output)
print(batchnorm_output.mean())

tensor([[-0.5587,  1.6940],
        [ 1.7318, -0.7548],
        [-0.5866, -0.7141],
        [-0.5866, -0.2252]], grad_fn=<NativeBatchNormBackward0>)
tensor(4.0978e-08, grad_fn=<MeanBackward0>)


In [44]:
mlp_layer = nn.Sequential(
    nn.Linear(5,2),
    nn.BatchNorm1d(2),
    nn.ReLU()
)

test_example = torch.randn(5,5) + 1 
print(test_example)
print(mlp_layer(test_example))

tensor([[-0.7427,  1.9736,  2.7683,  0.1659,  0.0054],
        [ 2.0893,  0.7640,  1.8830,  1.8220,  0.4925],
        [ 1.0571,  0.2857,  0.5241,  0.6252,  0.0909],
        [ 0.3798,  1.2828,  0.1494,  0.5730, -0.6616],
        [ 1.5438,  2.1906, -0.5518,  0.4498,  0.9726]])
tensor([[1.8739, 0.3665],
        [0.0000, 0.0000],
        [0.0000, 1.8093],
        [0.0000, 0.0000],
        [0.1695, 0.0000]], grad_fn=<ReluBackward0>)


## Optimization and Training Loop

In [46]:
import torch.optim as optim
adam_opt = optim.Adam(mlp_layer.parameters(), lr=1e-1)

/home/yash/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [49]:
training_example = torch.randn(100,5)
adam_opt.zero_grad()

cur_loss = torch.abs(1 - mlp_layer(training_example)).mean()
cur_loss.backward()
adam_opt.step()
print(cur_loss)

tensor(0.7652, grad_fn=<MeanBackward0>)


In [50]:
class ExampleModule(nn.Module):
    def __init__(self, input_dims, output_dims) -> None:
        super(ExampleModule, self).__init__()
        self.linear = nn.Linear(input_dims, output_dims)
        self.exponent = nn.Parameter(torch.tensor(1.))

    def forward(self, x):
        x = self.linear(x)
        x = x ** self.exponent
        return x

In [52]:
example_module = ExampleModule(10, 2)
print(list(example_module.parameters()))

[Parameter containing:
tensor(1., requires_grad=True), Parameter containing:
tensor([[ 0.1752,  0.1322, -0.2466,  0.2553,  0.3086,  0.0595,  0.0936, -0.0332,
         -0.0893, -0.0972],
        [-0.1721,  0.2393, -0.2147,  0.0393,  0.2266,  0.0535, -0.1146, -0.1469,
          0.2760,  0.0146]], requires_grad=True), Parameter containing:
tensor([-0.0087,  0.3104], requires_grad=True)]


In [53]:
print(list(example_module.named_parameters()))

[('exponent', Parameter containing:
tensor(1., requires_grad=True)), ('linear.weight', Parameter containing:
tensor([[ 0.1752,  0.1322, -0.2466,  0.2553,  0.3086,  0.0595,  0.0936, -0.0332,
         -0.0893, -0.0972],
        [-0.1721,  0.2393, -0.2147,  0.0393,  0.2266,  0.0535, -0.1146, -0.1469,
          0.2760,  0.0146]], requires_grad=True)), ('linear.bias', Parameter containing:
tensor([-0.0087,  0.3104], requires_grad=True))]
